# Resistance/Support AND Candles Patterns

In [1]:
import sys
sys.path.append('../')
import pandas as pd
from utilities.data_manager import ExchangeDataManager

exchange_name = "binance"
pair = "ETH/USDT"
coin = pair.split('/')[0]
tf = "1d"

exchange = ExchangeDataManager(
    exchange_name=exchange_name, 
    path_download="../database/exchanges"
)

df = exchange.load_data(
    coin=pair, 
    interval=tf
)

#Check if NA values are in data
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.head()

,open,high,low,close,volume
0,301.13,312.18,298.00,302.00,7030.71034
1,302.00,311.79,283.94,293.96,9537.84646
2,293.31,299.90,278.00,290.91,2146.19773
3,289.41,300.53,282.85,299.10,2510.13871
4,299.10,346.52,294.60,323.29,5219.44542


# Support and Resitance FUNCTIONS

In [2]:
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.low[i]>df1.low[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.low[i]<df1.low[i-1]):
            return 0
    return 1

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.high[i]<df1.high[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.high[i]>df1.high[i-1]):
            return 0
    return 1

In [3]:
length = len(df)
high = list(df['high'])
low = list(df['low'])
close = list(df['close'])
open = list(df['open'])
bodydiff = [0] * length

highdiff = [0] * length
lowdiff = [0] * length
ratio1 = [0] * length
ratio2 = [0] * length

def isEngulfing(l):
    row=l
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001      

    bodydiffmin = 0.002
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]<close[row-1] and
        open[row]>close[row] and 
        (open[row]-close[row-1])>=-0e-5 and close[row]<open[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]>close[row-1] and
        open[row]<close[row] and 
        (open[row]-close[row-1])<=+0e-5 and close[row]>open[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0
       
def isStar(l):
    bodydiffmin = 0.0020
    row=l
    highdiff[row] = high[row]-max(open[row],close[row])
    lowdiff[row] = min(open[row],close[row])-low[row]
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001
    ratio1[row] = highdiff[row]/bodydiff[row]
    ratio2[row] = lowdiff[row]/bodydiff[row]

    if (ratio1[row]>1 and lowdiff[row]<0.2*highdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]>close[row]):
        return 1
    elif (ratio2[row]>1 and highdiff[row]<0.2*lowdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]<close[row]):
        return 2
    else:
        return 0
    
def closeResistance(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.high[l]-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    c2 = abs(max(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    c3 = min(df.open[l],df.close[l])<min(levels, key=lambda x:abs(x-df.high[l]))
    c4 = df.low[l]<min(levels, key=lambda x:abs(x-df.high[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0
    
def closeSupport(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.low[l]-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c2 = abs(min(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c3 = max(df.open[l],df.close[l])>min(levels, key=lambda x:abs(x-df.low[l]))
    c4 = df.high[l]>min(levels, key=lambda x:abs(x-df.low[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0

In [4]:
n1=2
n2=2
backCandles=45
signal = [0] * length

for row in range(backCandles, len(df)-n2):
    ss = []
    rr = []
    for subrow in range(row-backCandles+n1, row+1):
        if support(df, subrow, n1, n2):
            ss.append(df.low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.high[subrow])
    #!!!! parameters
    if ((isEngulfing(row)==1 or isStar(row)==1) and closeResistance(row, rr, 150e-5) ):#and df.RSI[row]<30
        signal[row] = 1
    elif((isEngulfing(row)==2 or isStar(row)==2) and closeSupport(row, ss, 150e-5)):#and df.RSI[row]>70
        signal[row] = 2
    else:
        signal[row] = 0



In [5]:
df['signal']=signal

In [6]:
df[df['signal']==2].count()

open      41
high      41
low       41
close     41
volume    41
signal    41
dtype: int64

In [7]:
SLTPRatio = 1.5 #TP/SL Ratio
def mytarget(barsupfront, df1):
    length = len(df1)
    high = list(df1['high'])
    low = list(df1['low'])
    close = list(df1['close'])
    open = list(df1['open'])
    signal = list(df1['signal'])
    trendcat = [0] * length
    amount = [0] * length
    
    SL=0
    TP=0
    for line in range(backCandles, length-barsupfront-n2):

        if signal[line]==1:
            SL = max(high[line-1:line+1])#!!!!! parameters
            TP = close[line]-SLTPRatio*(SL-close[line])
            for i in range(1,barsupfront+1):
                if(low[line+i]<=TP and high[line+i]>=SL):
                    trendcat[line]=3
                    break
                elif (low[line+i]<=TP):
                    trendcat[line]=1 #win trend 1 in signal 1
                    amount[line]=close[line]-low[line+i]
                    break
                elif (high[line+i]>=SL):
                    trendcat[line]=2 #loss trend 2 in signal 1
                    amount[line]=close[line]-high[line+i]
                    break

        if signal[line]==2:
            SL = min(low[line-1:line+1])#!!!!! parameters
            TP = close[line]+SLTPRatio*(close[line]-SL)
    
            for i in range(1,barsupfront+1):
                if(high[line+i]>=TP and low[line+i]<=SL):
                    trendcat[line]=3
                    break
                elif (high[line+i]>=TP):
                    trendcat[line]=2 #win trend 2 in signal 2
                    amount[line]=high[line+i]-close[line]
                    break
                elif (low[line+i]<=SL):
                    trendcat[line]=1 #loss trend 1 in signal 2
                    amount[line]=low[line+i]-close[line]
                    break
    # return trendcat
    return amount

df['Trend'] = mytarget(16, df)
df['Amount'] = mytarget(16, df)

In [9]:
df[df['Amount']!=0]
df['Amount'].sum()/6

395.5100000000002

In [10]:
import numpy as np
conditions = [(df['Trend'] == 1) & (df['signal'] == 1), (df['Trend'] == 2) & (df['signal'] == 2)]
values = [1, 2]
df['result'] = np.select(conditions, values)

trendId=2
print(df[df['result']==trendId].result.count()/df[df['signal']==trendId].signal.count())
# df[ (df['Trend']!=trendId) & (df['Trend']!=3) & (df['signal']==trendId) ] # false positives

0.0


In [ ]:
dfpl = df[600:670]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.show()